In [1]:
# BertTopic using a pretrained model
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
import json
import os
from nltk.tokenize import sent_tokenize

! if not nbformat:
!    pip install --upgrade nbformat==4.2.0

/home/coco/git/lesswrong-alignment-trends/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


fish: Missing end to balance this if statement
 if not nbformat:
 ^^


In [2]:
with open(os.path.join(os.getcwd(), "./../lesswrong_data.json")) as f:
    json_data = json.load(f)["posts"]

preprocessed_corpus = []
preprocessed_tags = []

# using sentence trasnformers for embeddings:
# for post in json_data:
#     # Split into sentences for better context
#     sentences = sent_tokenize(post["text"])
#     post_tags = [post["tags"]] * len(sentences)

#     preprocessed_corpus.extend(sentences)
#     preprocessed_tags.extend(post_tags)

# assert len(preprocessed_corpus) == len(preprocessed_tags), "Number of sentences and tags must match"

# using the entite post as document: 
for post in json_data:
    preprocessed_corpus.append(post["text"])
    preprocessed_tags.append(post["tags"])

print(preprocessed_tags)
print(len(preprocessed_corpus), len(preprocessed_tags))

[['AI'], ['AI'], ['Interpretability (ML & AI)', 'Language Models (LLMs)', 'AI'], ['AI'], ['AI Governance', 'AI Timelines', 'AI'], ['AI Governance', 'Careers', 'AI'], ['OpenAI', 'Recursive Self-Improvement', 'Reinforcement learning', 'Superintelligence', 'AI'], ['AI'], ['AI'], ['Cognitive Science', 'Forecasting & Prediction', 'Utility Functions', 'AI', 'Rationality'], ['AI'], ['Newsletters', 'AI'], ['AI'], ['AI Control', 'AI'], ['AI Alignment Fieldbuilding', 'Philanthropy / Grant making (Topic)', 'AI'], ['Humor', 'Journalism', 'AI'], ['Simulator Theory', 'AI'], ['AI Control', 'DeepMind', 'Economic Consequences of AGI', 'Financial Investing', 'AI'], ['AI Safety Public Materials', 'AI'], ['Newsletters', 'AI'], ['AI Governance', 'AI'], ['Outer Alignment', 'Aligned AI Proposals', 'Gradient Descent', 'Reinforcement learning', 'AI-Assisted Alignment', 'Inner Alignment', 'Value Learning', 'AI'], ['AI'], ['AI'], ['AI Evaluations', 'Introspection', 'Language Models (LLMs)', 'Situational Awarenes

In [3]:
# get unique tags
unique_tags = []
for tags in preprocessed_tags:
    for tag in tags:
        if tag not in unique_tags:
            unique_tags.append(tag)
print(unique_tags)

ctfidf_model = ClassTfidfTransformer(
    seed_words=unique_tags,
    seed_multiplier=5,
    reduce_frequent_words=True
)

['AI', 'Interpretability (ML & AI)', 'Language Models (LLMs)', 'AI Governance', 'AI Timelines', 'Careers', 'OpenAI', 'Recursive Self-Improvement', 'Reinforcement learning', 'Superintelligence', 'Cognitive Science', 'Forecasting & Prediction', 'Utility Functions', 'Rationality', 'Newsletters', 'AI Control', 'AI Alignment Fieldbuilding', 'Philanthropy / Grant making (Topic)', 'Humor', 'Journalism', 'Simulator Theory', 'DeepMind', 'Economic Consequences of AGI', 'Financial Investing', 'AI Safety Public Materials', 'Outer Alignment', 'Aligned AI Proposals', 'Gradient Descent', 'AI-Assisted Alignment', 'Inner Alignment', 'Value Learning', 'AI Evaluations', 'Introspection', 'Situational Awareness', 'AI Alignment Intro Materials', 'Orthogonality Thesis', 'Goal-Directedness', 'Category theory', 'Fixed Point Theorems', 'Formal Proof', 'Gödelian Logic', "Löb's theorem", 'Logic & Mathematics ', 'Logical Uncertainty', 'World Modeling', 'AI Takeoff', 'Consciousness', 'Neuroscience', 'Predictive Pro

In [4]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

topic_model = BERTopic(
    embedding_model=embedding_model,
    min_topic_size=3,
    nr_topics="none",
    ctfidf_model=ctfidf_model,
    verbose=True
)

topics, probs = topic_model.fit_transform(preprocessed_corpus)


2025-06-16 17:05:23,703 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 7/7 [00:06<00:00,  1.11it/s]
2025-06-16 17:05:30,373 - BERTopic - Embedding - Completed ✓
2025-06-16 17:05:30,374 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
/home/coco/git/lesswrong-alignment-trends/venv/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
2025-06-16 17:05:39,338 - BERTopic - Dimensionality - Completed ✓
2025-06-16 17:05:39,339 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-06-16 17:05:39,359 - BERTopic - Cluster - Completed ✓
2025-06-16 17:05:39,360 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-06-16 17:05:39,874 - BERTopic - Representati

In [5]:
hierarchical_topics = topic_model.hierarchical_topics(preprocessed_corpus)
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 17/17 [00:00<00:00, 204.84it/s]


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [llms_game_delegate_concepts_answer, , ,
                       elicitation_site_evals_results_ads],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAAdx9aLSR3mPx3H1otJHeY/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAACAQcAAAAAAAIBBwAAAAAAAgEbAAAAAAACARsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [simulator_reward_characters_hacking_simulators, , ,
                       elicitation_site_results_evals_ads],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAABKfxmyrmrmP0p/GbKuauY/HcfWi0kd5j8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAOcAAAAAAAAA5wAAAAAAAAETAAAAAAAAARMA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [feedback_untrusted_trusted_editing_monitoring, , ,
                       results_simulator_elicitation_site_model],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAB0LHI9lXvnP3Qscj2Ve+c/Sn8Zsq5q5j8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAALsAAAAAAAAAuwAAAAAAAQEDAAAAAAABAQMA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [results_feedback_monitoring_model_untrusted, , ,
                       agents_village_zochi_their_gpt],
              'type': 'scatter',
              'x': {'bdata': 'dCxyPZV75z8x066vqqHoPzHTrq+qoeg/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAADAN8AAAAAAAMA3wAAAAAAAgEvAAAAAAACAS8A=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [steering_vector_embeddings_spatial_vectors, , ,
                       results_feedback_monitoring_model_untrusted],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAA2eyWzGbLpPzZ7JbMZsuk/MdOur6qh6D8=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAAAAFMAAAAAAAAAUwAAAAAAAsEPAAAAAAACwQ8A=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [data_onionai_long_his_tokens, , ,
                       opus_claude_anthropic_it_do],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAACGEpxDCNfeP4YSnEMI194/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAABAVcAAAAAAAEBVwAAAAAAAwFfAAAAAAADAV8A=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [opus_claude_it_anthropic_but, , ,
                       advisor_01_principal_alignment_sharkey],
              'type': 'scatter',
              'x': {'bdata': 'hhKcQwjX3j+XOrtbt2bgP5c6u1u3ZuA/AAAAAAAAAAA=', 'dtype': 'f8'},
              'xaxis': 'x',
              'y': {'bdata': 'AAAAAACAVsAAAAAAAIBWwAAAAAAAQFrAAAAAAABAWsA=', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hoverinfo': 'text',
              'marker': {'color': 'rgb(61,153,112)'},
              'mode': 'lines',
              'text': [wishes_wish_all_jobs_ai, , , you_it_opus_is_this],
              'type': 'scatter',
              'x': {'bdata': 'AAAAAAAAAAD+PZBl/MniP/49kGX8yeI/lzq7W7dm4D8=', 'dtype': 'f8'},
  

In [6]:
topic_model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'advisor | 01 | principal | alignment | sharkey', 30],
                                   [1, 'data | onionai | long | his | tokens', 25],
                                   [2, 'technical | governance | ai | policy | work', 20],
                                   [3, 'opus | claude | anthropic | it | do', 16],
                                   [4, 'material | variables | day | lunch | convenience', 13],
                                   [5, 'estimates | scorecard | hours | costs | compliance', 13],
                                   [6, 'wishes | wish | all | jobs | ai', 10],
                                   [7, 'elicitation | site | evals | results | ads', 9],
                                   [8, 'steering | vector | embeddings | spatial | vectors', 8],
                                   [9, 'simulator | reward | characters | hacking | simulators', 8],
                                   [10, 'advocacy | political | voting | regulations | bill', 8],
                                   [11, 'polysemanticity | networks | superposition | claim | solutions',
                                    7],
                                   [12, 'advocacy | donate | donating | grantmaker | soares', 7],
                                   [13, 'naming | named | mommunes | names | reddit', 7],
                                   [14, 'tex | font | mjxc | family | src', 6],
                                   [15, 'agents | village | zochi | their | gpt', 6],
                                   [16, 'llms | game | delegate | concepts | answer', 4],
                                   [17, 'feedback | untrusted | trusted | editing | monitoring', 4]],
                                  dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': 'HhkUEA0NCgkICAgHBwcGBgQE', 'dtype': 'i1'},
                         'sizemode': 'area',
                         'sizeref': 0.01875,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': ('Gn/PwHRSvMCIsxhBuyIVQV2jHEG858' ... 'IfQXIQHUH0DbvAxJ+xwH/1zsAeBATA'),
                    'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': ('vDGEP5RMGUBrCrpA9HCkQE+FpUCsJS' ... 'aHQIQBwUCeU4g/bWcvQH+Zi70+6kpB'),
                    'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(-7.4569054),
                                'y': np.float32(7.33411),
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': np.float32(2.0650415),
                                'xshift': 10,
                                'y': np.float32(14.843059)}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(2.0650415),
                           'x1': np.float32(2.0650415),
                           'y0': np.float32(-0.17483906),
                           'y1': np.float32(14.843059)},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(-7

In [7]:
topic_model.visualize_topics(use_ctfidf=True)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'advisor | 01 | principal | alignment | sharkey', 30],
                                   [1, 'data | onionai | long | his | tokens', 25],
                                   [2, 'technical | governance | ai | policy | work', 20],
                                   [3, 'opus | claude | anthropic | it | do', 16],
                                   [4, 'material | variables | day | lunch | convenience', 13],
                                   [5, 'estimates | scorecard | hours | costs | compliance', 13],
                                   [6, 'wishes | wish | all | jobs | ai', 10],
                                   [7, 'elicitation | site | evals | results | ads', 9],
                                   [8, 'steering | vector | embeddings | spatial | vectors', 8],
                                   [9, 'simulator | reward | characters | hacking | simulators', 8],
                                   [10, 'advocacy | political | voting | regulations | bill', 8],
                                   [11, 'polysemanticity | networks | superposition | claim | solutions',
                                    7],
                                   [12, 'advocacy | donate | donating | grantmaker | soares', 7],
                                   [13, 'naming | named | mommunes | names | reddit', 7],
                                   [14, 'tex | font | mjxc | family | src', 6],
                                   [15, 'agents | village | zochi | their | gpt', 6],
                                   [16, 'llms | game | delegate | concepts | answer', 4],
                                   [17, 'feedback | untrusted | trusted | editing | monitoring', 4]],
                                  dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': 'HhkUEA0NCgkICAgHBwcGBgQE', 'dtype': 'i1'},
                         'sizemode': 'area',
                         'sizeref': 0.01875,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': ('telGP2jNrj6o31a+YiRDPnYIrL/bgb' ... '1uP5tuPz+9YNq/vNq5QFGks0AcoqdA'),
                    'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': ('JlyHwFWZfcCRHl/AVzGKwOeIcMCWik' ... 'DKQJaCxEBcv4HATgcMQe9SF0EtqSBB'),
                    'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(-1.9619888),
                                'y': np.float32(3.4096963),
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': np.float32(2.3585775),
                                'xshift': 10,
                                'y': np.float32(11.785694)}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(2.3585775),
                           'x1': np.float32(2.3585775),
                           'y0': np.float32(-4.9663014),
                           'y1': np.float32(11.785694)},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(-

In [8]:
topic_model.visualize_heatmap()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_advisor_01_principal, 1_data_onionai_long,
                    2_technical_governance_ai, 3_opus_claude_anthropic,
                    4_material_variables_day, 5_estimates_scorecard_hours,
                    6_wishes_wish_all, 7_elicitation_site_evals,
                    8_steering_vector_embeddings, 9_simulator_reward_characters,
                    10_advocacy_political_voting, 11_polysemanticity_networks...,
                    12_advocacy_donate_donating, 13_naming_named_mommunes,
                    14_tex_font_mjxc, 15_agents_village_zochi,
                    16_llms_game_delegate, 17_feedback_untrusted_trusted],
              'xaxis': 'x',
              'y': [0_advisor_01_principal, 1_data_onionai_long,
                    2_technical_governance_ai, 3_opus_claude_anthropic,
                    4_material_variables_day, 5_estimates_scorecard_hours,
                    6_wishes_wish_all, 7_elicitation_site_evals,
                    8_steering_vector_embeddings, 9_simulator_reward_characters,
                    10_advocacy_political_voting, 11_polysemanticity_networks...,
                    12_advocacy_donate_donating, 13_naming_named_mommunes,
                    14_tex_font_mjxc, 15_agents_village_zochi,
                    16_llms_game_delegate, 17_feedback_untrusted_trusted],
              'yaxis': 'y',
              'z': {'bdata': ('AgAAAAAA8D+4sj1lJu7hP9ATa357CN' ... '98CH+41D9g8dR6crbTPwIAAAAAAPA/'),
                    'dtype': 'f8',
                    'shape': '18, 18'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': 'Trend'}},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Similarity Matrix</b>',
                         'x': 0.55,
                         'xanchor': 'center',
                         'y': 0.95,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})

In [9]:
# let's save the model 
model_path = os.path.join(os.getcwd(), "bert_topic_model.pkl")
topic_model.save(model_path, "pickle")

2025-06-16 17:09:08,997 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [11]:
# Example loading and using the model to predict a new post
pickle_in = open(model_path, "rb")
model = BERTopic.load(model_path)

pred_topics, pred_probs = model.transform("This random sentence is about steering and embeddings and I'll also mention vectors. Let's see if it gets classified into the right topic.")

Batches: 100%|██████████| 1/1 [00:00<00:00,  2.98it/s]
2025-06-16 17:10:16,281 - BERTopic - Dimensionality - Reducing dimensionality of input embeddings.
2025-06-16 17:10:19,386 - BERTopic - Dimensionality - Completed ✓
2025-06-16 17:10:19,390 - BERTopic - Clustering - Approximating new points with `hdbscan_model`
2025-06-16 17:10:19,391 - BERTopic - Cluster - Completed ✓


In [23]:
topic_model.get_topic(pred_topics[0].item())

[('steering', np.float64(0.6208019693970465)),
 ('vector', np.float64(0.5459278918864175)),
 ('embeddings', np.float64(0.535821876416429)),
 ('spatial', np.float64(0.5314622593474108)),
 ('vectors', np.float64(0.527974331521925)),
 ('universal', np.float64(0.5018802529315438)),
 ('text', np.float64(0.4977924093622967)),
 ('arxiv', np.float64(0.47312448313248767)),
 ('visual', np.float64(0.46977128238787924)),
 ('representation', np.float64(0.45545537808466063))]